In [ ]:
import numpy as np
import pandas as pd
#import sklearn
import tensorflow as tf
#import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow.keras.losses import MSE, KLD
import phenograph

In [ ]:
!pip install phenograph

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.layers import Input, Dense

from keras.models import Model

from keras import regularizers

#from keras.datasets import mnist

import numpy as np

import matplotlib.pyplot as plt

from sklearn.manifold import TSNE

In [ ]:
def myscatter(Y, class_idxs, legend=False, ran=True, seed=229):
    if ran:
        np.random.seed(seed)
    Y = np.array(Y)
    fig, ax = plt.subplots(figsize=(6,4), dpi=300)
    classes = list(np.unique(class_idxs))
    markers = 'osD' * len(classes)
    colors = plt.cm.rainbow(np.linspace(0, 1, len(classes)))
    if ran:
        np.random.shuffle(colors)

    for i, cls in enumerate(classes):
        mark = markers[i]
        ax.plot(Y[class_idxs == cls, 0], Y[class_idxs == cls, 1], marker=mark,
                linestyle='', ms=4, label=str(cls), alpha=1, color=colors[i],
                markeredgecolor='black', markeredgewidth=0.15)
    if legend:
        ax.legend(bbox_to_anchor=(1.03, 1), loc=2, borderaxespad=0, fontsize=10, markerscale=2, frameon=False,
                  ncol=2, handletextpad=0.1, columnspacing=0.5)

    plt.xticks([])
    plt.yticks([])

    return ax

In [ ]:
def dotsne(X, dim=2, ran=23):
    tsne = TSNE(n_components=dim, random_state=ran)
    Y_tsne = tsne.fit_transform(X)
    return Y_tsne

In [ ]:
count = np.load('/content/drive/My Drive/CompBio/test_data/count.npy')
idents = np.load('/content/drive/My Drive/CompBio/test_data/idents.npy')
#count = np.array(pd.read_csv("real_raw.csv", index_col=0, sep="\t"))
count = np.transpose(count)

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/idents1.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/data1.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.load('idents.npy')
count = np.array(pd.read_csv("pbmc_raw.csv", index_col=0, sep="\t"))

In [ ]:
count = np.array(pd.read_csv("demo_counts.csv", index_col=0, sep="\t"))
idents = np.load('idents.npy')

In [ ]:
idents = np.array(pd.read_csv("ileum_cell_info.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("ileum_raw.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/ileum_cell_info_SCT.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/ileum_SCT.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("colon_cell_info.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("colon_raw.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/colon_cell_info_SCT.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/colon_SCT.csv", index_col=0, sep="\t"))

In [ ]:
idents = np.load('idents.npy')
count = np.array(pd.read_csv("breast_cancer.csv", index_col=0, sep="\t"))

In [ ]:
count = np.array(pd.read_csv("drug_response_normalize.csv", index_col=0, sep=","))

In [ ]:
idents = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/BM4_select_SCT_labels.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/BM4_select_SCT.csv", index_col=0, sep="\t"))

In [ ]:
# read data
idents = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/GSM3587923_AML1012-D0_SCT_labels.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/GSM3587923_AML1012-D0_SCT.csv", index_col=0, sep="\t"))

In [ ]:
# read data
idents = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/AML707B-all_celltype_select.csv", index_col=0, sep="\t"))
#cells_number = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/GSM3587969_AML707B-D0_info_number.csv", index_col=0, sep="\t"))
count = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/AML707B-all_normalize_select.csv", index_col=0, sep="\t"))

In [ ]:
count = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/pbmc_SCT.csv", index_col=0, sep="\t"))
idents = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/pbmc_idents.csv", index_col=0, sep="\t"))

In [ ]:
count = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/real_data.csv", index_col=0, sep="\t"))

In [ ]:
count = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/normal_pancreas.csv", index_col=0, sep="\t"))
idents = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/idents_normal_pancreas.csv", index_col=0, sep="\t"))

In [ ]:
count = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/IPMN_data.csv", index_col=0, sep="\t"))
idents = np.array(pd.read_csv("/content/drive/My Drive/CompBio/test_data/idents_IPMN.csv", index_col=0, sep="\t"))

In [ ]:
from sklearn import decomposition

In [ ]:
idents = idents.flatten()

In [ ]:
[n_sample, n_gene] = count.shape

In [ ]:
x_train = count.astype('float32')

In [ ]:
idents = idents.astype('str')

In [ ]:
idents = np.zeros(n_sample)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
def clustering(h, n_cluster, k=15, f="kmeans"):
    if f == "kmeans":
        labels = KMeans(n_clusters=n_cluster, random_state=0).fit(h).labels_
    elif f == "spectral":
        labels = SpectralClustering(n_clusters=n_cluster, affinity="precomputed", assign_labels="discretize",
                                    random_state=0).fit_predict(adj)
    
    return labels

In [ ]:
from sklearn.neighbors import kneighbors_graph
def find_neighbors(X):
    A = kneighbors_graph(X, 10, mode="connectivity", metric="euclidean", include_self=True)
    return A.toarray()

In [ ]:
import keras
class DenseTranspose(keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias", initializer="zeros",
                                      shape=[self.dense.input_shape[-1]])
        super().build(batch_input_shape)
    def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)
        return self.activation(z + self.biases)

In [ ]:
encoded1 = Dense(512, activation = "relu", kernel_constraint = keras.constraints.NonNeg(), input_shape=(n_gene,), use_bias=True,
                 kernel_initializer='glorot_uniform',bias_initializer='zeros')

encoded2 = Dense(256, activation = "relu", activity_regularizer = regularizers.l1(10e-4), input_shape=(512,), use_bias=True,
                 kernel_initializer='glorot_uniform',bias_initializer='zeros')

encoded3 = Dense(128, activation = "relu", activity_regularizer = regularizers.l1(10e-4), input_shape=(256,), use_bias=True,
                 kernel_initializer='glorot_uniform',bias_initializer='zeros')

encoded4 = Dense(25, activation = "relu", activity_regularizer = regularizers.l1(10e-4), input_shape=(128,), use_bias=True,
                 kernel_initializer='glorot_uniform',bias_initializer='zeros')

encoder = keras.models.Sequential([encoded1, encoded2, encoded3, encoded4])
encoder1 = keras.models.Sequential([encoded1])
encoder2 = keras.models.Sequential([encoded2])
encoder3 = keras.models.Sequential([encoded3])
encoder4 = keras.models.Sequential([encoded4])

In [ ]:
decoded1 = DenseTranspose(encoded4, activation = "relu")

decoded2 = DenseTranspose(encoded3, activation = "relu")

decoded3 = DenseTranspose(encoded2, activation = "relu")

decoded4 = DenseTranspose(encoded1, activation = "relu")

decoder = keras.models.Sequential([decoded1, decoded2, decoded3, decoded4])
decoder1 = keras.models.Sequential([decoded1])
decoder2 = keras.models.Sequential([decoded2])
decoder3 = keras.models.Sequential([decoded3])
decoder4 = keras.models.Sequential([decoded4])

In [ ]:
# no
decoded1 = Dense(128, activation = "relu", activity_regularizer = regularizers.l1(10e-4), input_shape=(25,), use_bias=True)

decoded2 = Dense(256, activation = "relu", activity_regularizer = regularizers.l1(10e-4), input_shape=(128,), use_bias=True)

decoded3 = Dense(512, activation = "relu", activity_regularizer = regularizers.l1(10e-4), input_shape=(256,), use_bias=True)

decoded4 = Dense(n_gene, activation = "relu", activity_regularizer = regularizers.l1(10e-4), input_shape=(512,), use_bias=True)

decoder = keras.models.Sequential([decoded1, decoded2, decoded3, decoded4])
decoder1 = keras.models.Sequential([decoded1])
decoder2 = keras.models.Sequential([decoded2])
decoder3 = keras.models.Sequential([decoded3])
decoder4 = keras.models.Sequential([decoded4])

In [ ]:
autoencoder1 = keras.models.Sequential([encoded1, decoded4])
autoencoder2 = keras.models.Sequential([encoded2, decoded3])
autoencoder3 = keras.models.Sequential([encoded3, decoded2])
autoencoder4 = keras.models.Sequential([encoded4, decoded1])

In [ ]:
# no
h = autoencoder.ec(x_train)
h1 = autoencoder.ec1(x_train)
h1 = autoencoder.ec2(h1)
centers, labels = get_centers(np.array(h))
centers1, labels1 = get_centers(np.array(h1))
q = cal_cluster(h, centers, 1.0)
q1 = cal_cluster(h1, centers1, 1.0)
tf.reduce_mean(KLD(q, q1))

In [ ]:
labels, _,  _ = phenograph.cluster(np.array(h))
print(labels[0:50])
labels, _,  _ = phenograph.cluster(np.array(h1))
print(labels[0:50])

In [ ]:
from tensorflow.keras import activations, constraints, initializers, regularizers
from tensorflow.keras.layers import Layer, InputSpec
import tensorflow.keras.backend as K

class ClusteringLayer(Layer):
    def __init__(self, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=None, initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def target_distribution(q):
    q = q.numpy()
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [ ]:
def cal_cluster(hidden, clusters, alpha):
    clusters = tf.convert_to_tensor(clusters, dtype=tf.float32)
    q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(hidden, axis=1) - clusters), axis=2) / alpha))
    q **= (alpha + 1.0) / 2.0
    q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
    return q

In [ ]:
def cal_dist(hidden, clusters):
    clusters = tf.convert_to_tensor(clusters, dtype=tf.float32)
    dist1 = K.sum(K.square(K.expand_dims(hidden, axis=1) - clusters), axis=2)
    temp_dist1 = dist1 - tf.reshape(tf.reduce_min(dist1, axis=1), [-1, 1])
    temp_dist1 = K.transpose(K.transpose(temp_dist1) / K.max(temp_dist1, axis=1))
    q = K.exp(-temp_dist1 * 10)
    #q = K.exp(-temp_dist1)
    q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
    q = K.pow(q, 2)
    q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
    dist2 = dist1 * q
    return dist1, dist2

In [ ]:
def cal_latent(hidden, alpha):
    sum_y = K.sum(K.square(hidden), axis=1)
    num = -2.0 * tf.matmul(hidden, tf.transpose(hidden)) + tf.reshape(sum_y, [-1, 1]) + sum_y
    num = num / alpha
    num = tf.pow(1.0 + num, -(alpha + 1.0) / 2.0)
    zerodiag_num = num - tf.linalg.diag(tf.linalg.diag_part(num))
    latent_p = K.transpose(K.transpose(zerodiag_num) / K.sum(zerodiag_num, axis=1))
    return num, latent_p

def target_dis(latent_p):
    latent_q = tf.transpose(tf.transpose(tf.pow(latent_p, 2)) / tf.reduce_sum(latent_p, axis = 1))
    return tf.transpose(tf.transpose(latent_q) / tf.reduce_sum(latent_q, axis = 1))

In [ ]:
def computeCentroids(data, labels):
    n_clusters = len(np.unique(labels))
    return np.array([data[labels == i].mean(0) for i in range(n_clusters)])

def get_centers(Y):
    #from sklearn.cluster import SpectralClustering
    l, _,  _ = phenograph.cluster(Y)
    #l = KMeans(n_clusters=10, random_state=0).fit(Y).labels_
    centers = computeCentroids(Y, l)
    return centers, l

In [ ]:
clustering_layer = ClusteringLayer(name='clustering', input_shape=(25,))
clustering = keras.models.Sequential([clustering_layer])

In [ ]:
class AE(tf.keras.Model):

    def __init__(self, encoder, decoder, clustering):
        super(AE, self).__init__()
        self.ec = encoder
        self.dc = decoder
        self.clustering = clustering

In [ ]:
from tensorflow.keras.losses import MSE, KLD
import math

class SAE():

    def __init__(self, X):
        super(SAE, self).__init__()
        self.X = X
        self.ec1 = encoder1
        self.ec2 = encoder2
        self.ec3 = encoder3
        self.ec4 = encoder4
        self.dc1 = decoder1
        self.dc2 = decoder2
        self.dc3 = decoder3
        self.dc4 = decoder4
        self.autoencoder1 = autoencoder1
        self.autoencoder2 = autoencoder2
        self.autoencoder3 = autoencoder3
        self.autoencoder4 = autoencoder4
        self.ec = encoder
        self.dc = decoder
        self.autoencoder = keras.models.Sequential([encoder, decoder])
        self.clustering = clustering
        self.AE = AE(encoder, decoder, clustering)
        
    def train1(self):
        optimizer = tf.keras.optimizers.Adam(lr=0.002)
        for epoch in range(0, 100):
            with tf.GradientTape(persistent=True) as tape:
                h = self.ec1(self.X)
                z = self.dc4(h)
                #im = self.im(z)
                
                #impute_out = tf.multiply(self.I, im)
                #impute_out = preprocessing.scale(impute_out)
                
                
                #count1 = tf.multiply(self.I, self.X)
                #z1 = tf.multiply(self.I, z)
                
                loss = tf.reduce_mean(MSE(count, z))
                
                #if epoch % 5 == 0:
                    #A = np.nonzero(find_neighbors(h))
                    #Ax = A[0]
                    #Ay = A[1]
                    #A = np.zeros([n_sample, n_sample])
                    #A[Ax, Ay] = 1
                    #score = measure_matrix(A0, A, n_sample)
                    #loss = loss + math.exp(0-score) / 2

            vars = self.autoencoder1.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 10 == 0:
                print(loss)
        print("Finish!")
        #print(self.ec1.weights)
        #print(self.dc4.weights)
        #print(self.autoencoder1.weights)
        return h
        
    def train2(self, h):
        optimizer = tf.keras.optimizers.Adam(lr=0.005)
        c = h
        #c = preprocessing.scale(np.array(h))
        for epoch in range(0, 120):
            with tf.GradientTape(persistent=True) as tape:
                h1 = self.ec2(h)
                z = self.dc3(h1)
                loss = tf.reduce_mean(MSE(c, z))
                

            vars = self.autoencoder2.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 50 == 0:
                print(loss)
        print("Finish!")
        #print(self.ec1.weights)
        return h1
    
    def train3(self, h):
        optimizer = tf.keras.optimizers.Adam(lr=0.01)
        c = h
        #c = preprocessing.scale(np.array(h))
        for epoch in range(0, 350):
            with tf.GradientTape(persistent=True) as tape:
                h1 = self.ec3(h)
                z = self.dc2(h1)
                loss = tf.reduce_mean(MSE(c, z))

            vars = self.autoencoder3.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 50 == 0:
                print(loss)
        print("Finish!")
        return h1
        
    def train4(self, h):
        optimizer = tf.keras.optimizers.Adam(lr=0.01)
        c = h
        #c = preprocessing.scale(np.array(h))
        for epoch in range(0, 350):
            with tf.GradientTape(persistent=True) as tape:
                h1 = self.ec4(h)
                z = self.dc1(h1)
                loss = tf.reduce_mean(MSE(c, z))

            vars = self.autoencoder4.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 50 == 0:
                print(loss)
        print("Finish!")
        return h1

    def train(self):

        optimizer = tf.keras.optimizers.Adam()
        for epoch in range(0, 100):
            
            with tf.GradientTape(persistent=True) as tape:
                h = self.ec(self.X)
                z = self.dc(h)
                loss = tf.reduce_mean(MSE(count, z))
                
            vars = self.AE.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 10 == 0:
                print(loss)
                #print(p)
                #print(q_out)
        print("Finish!")
    
    def clustering_train(self):
        #print(self.ec1.weights)
        h = self.ec(self.X)
        h1 = self.ec1(self.X)
        h1 = self.ec2(h1)
        centers, labels = get_centers(np.array(h))
        centers1, labels1 = get_centers(np.array(h1))

        print(h.shape)
        print(h1.shape)
        
        #self.clustering.get_layer(name='clustering').clusters = centers
        #q = self.clustering(h)
        #p = target_distribution(q)

        q = cal_cluster(h, centers, 1.0)
        p = target_distribution(q)
        q1 = cal_cluster(h1, centers1, 1.0)
        optimizer = tf.keras.optimizers.Adam()
        for epoch in range(1, 51):
            if epoch % 10 == 0:
                centers, labels = get_centers(np.array(h))
                centers1, labels1 = get_centers(np.array(h1))
                #self.clustering.get_layer(name='clustering').clusters = centers
                #q = self.clustering(h)
                q = cal_cluster(h, centers, 1.0)
                p = target_distribution(q)
                q1 = cal_cluster(h1, centers1, 1.0)
                #print(clustering_layer.clusters)
            with tf.GradientTape(persistent=True) as tape:
                h = self.ec(self.X)
                z = self.dc(h)
                loss = tf.reduce_mean(MSE(count, z))
                
                #q_out = self.clustering(h)
                q_out = cal_cluster(h, centers, 1.0)
                cluster_loss = tf.reduce_mean(KLD(q_out, p))
                loss = loss + 2.5 * cluster_loss

                #centers, labels = get_centers(np.array(h))
                latent_dist1, latent_dist2 = cal_dist(h, centers)
                kmeans_loss = tf.reduce_mean(tf.reduce_sum(latent_dist2, axis=1))
                loss = loss + 0.1 * kmeans_loss

                num, latent_p = cal_latent(h, 1)
                latent_q = target_dis(latent_p)
                latent_p = latent_p + tf.linalg.diag(tf.linalg.diag_part(num))
                latent_q = latent_q + tf.linalg.diag(tf.linalg.diag_part(num))
                #cross_entropy = -tf.reduce_sum(latent_q * tf.math.log(latent_p))
                #entropy = -tf.reduce_sum(latent_q * tf.math.log(latent_q))
                #kl_loss = cross_entropy - entropy
                kl_loss = tf.reduce_mean(KLD(latent_p, latent_q))
                loss = loss + 2.5 * kl_loss

                h1 = self.ec1(self.X)
                h1 = self.ec2(h1)
                q1 = cal_cluster(h1, centers1, 1.0)
                if q.shape == q1.shape:
                    cluster_loss_1 = tf.reduce_mean(KLD(q_out, q1))
                    loss = loss + 2.5 * cluster_loss_1
                else:
                    cluster_loss_1 = 0


            vars = self.autoencoder4.trainable_weights
            #vars = self.AE.trainable_weights
            grads = tape.gradient(loss, vars)
            optimizer.apply_gradients(zip(grads, vars))
            if epoch % 10 == 0:
                print(loss)
                print(cluster_loss)
                print(kmeans_loss)
                print(kl_loss)
                print(cluster_loss_1)
                #print(latent_dist2)
                #print(latent_dist2.shape)
                #print(clustering_layer.clusters)
                #print(clustering_layer.weights)
                #print(p)
                #print(q_out)
        print("Finish!")
        

In [ ]:
autoencoder = SAE(x_train)

In [ ]:
h = autoencoder.train1()

In [ ]:
h = autoencoder.train2(h)

In [ ]:
h = autoencoder.train3(h)

In [ ]:
h = autoencoder.train4(h)

In [ ]:
autoencoder.train()

In [ ]:
autoencoder.clustering_train()

In [ ]:
encoded_data = autoencoder.ec(x_train)
ed = np.array(encoded_data)
ed = dotsne(ed)

In [ ]:
myscatter(ed, idents, legend=True)

In [ ]:
#centers, labels = get_centers(np.array(encoded_data))
dist1 = K.sum(K.square(K.expand_dims(encoded_data, axis=1) - centers), axis=2)
temp_dist1 = dist1 - tf.reshape(tf.reduce_min(dist1, axis=1), [-1, 1])
temp_dist1 = K.transpose(K.transpose(temp_dist1) / K.max(temp_dist1, axis=1))
q = K.exp(-temp_dist1 * 10)
q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
q = K.pow(q, 2)
q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
dist2 = dist1 * q
kmeans_loss = tf.reduce_mean(tf.reduce_sum(dist2, axis=1))
kmeans_loss

In [ ]:
q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(encoded_data, axis=1) - centers), axis=2) / 1))
q **= (1 + 1.0) / 2.0
q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
q[1]

In [ ]:
num, latent_p = cal_latent(encoded_data, 1)
latent_p = latent_p + tf.linalg.diag(tf.linalg.diag_part(num))
latent_p[0:100,8]

In [ ]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, homogeneity_score, completeness_score
def measure(true, pred):
    NMI = round(normalized_mutual_info_score(true, pred), 2)
    RAND = round(adjusted_rand_score(true, pred), 2)
    HOMO = round(homogeneity_score(true, pred), 2)
    COMP = round(completeness_score(true, pred), 2)
    return [NMI, RAND, HOMO, COMP]

In [ ]:
labels, _,  _ = phenograph.cluster(np.array(encoded_data))
measure(idents, labels)

In [ ]:
myscatter(ed,labels, legend=True)

In [ ]:
labels = KMeans(n_clusters=12, random_state=0).fit(ed).labels_
measure(idents, labels)

In [ ]:
myscatter(ed,labels, legend=True)

In [ ]:
ed = dotsne(count)
myscatter(ed, idents, legend=True)

In [ ]:
pca = decomposition.PCA(n_components = 2)
pca.fit(count)
count_pca = pca.transform(count)

In [ ]:
myscatter(count_pca, idents, legend=True)